In [1]:
from ultralytics import YOLO
import cv2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


In [3]:
import os
import pandas as pd

DATASET_DIR = '../datasets/custom/'

# Function to get list of video clips and their labels
def get_video_clips_and_labels(dataset_dir):
    data = {'clip_path': [], 'label': []}
    
    for root, dirs, files in os.walk(dataset_dir):
        for dir_name in dirs:
            label = dir_name
            clips_dir = os.path.join(root, dir_name)
            
            for clip_file in os.listdir(clips_dir):
                clip_path = os.path.join(clips_dir, clip_file)
                data['clip_path'].append(clip_path)
                data['label'].append(label)
    
    return data

# Create DataFrame and save to CSV
data = get_video_clips_and_labels(DATASET_DIR)
df = pd.DataFrame(data)
csv_path = os.path.join(DATASET_DIR, 'dataset.csv')
df.to_csv(csv_path, index=False)

print(f"CSV file saved at: {csv_path}")


CSV file saved at: ../datasets/custom/dataset.csv


In [4]:
df = pd.read_csv('../datasets/custom/dataset.csv')

In [5]:
df_walking = df[df['label']=='walking'].reset_index()
df_jogging = df[df['label']=='jogging'].reset_index()

In [6]:
model = YOLO('models/yolov8n-pose.pt') 

In [9]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=3400)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()
for train_df in [df_jogging]:
    label = train_df['label'][0]
    print(f"Collecting: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': label})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

Collecting: jogging


Processing Videos: 100%|██████████| 1/1 [02:25<00:00, 145.88s/it]

Skipped: 0


In [10]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=11000)

for train_df in [df_walking]:
    label = train_df['label'][0]
    print(f"Collecting: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': label})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

Collecting: walking


Processing Videos:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Videos: 100%|██████████| 1/1 [07:56<00:00, 476.82s/it]


Skipped: 0


In [ ]:
keypoints_df

In [11]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/custom_dataset/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)